# NEXRAD data

Is it raining in Miami?

The Next Generation Weather Radar (NEXRAD) system is a network of 160 high-resolution S-band Doppler weather radars jointly operated by the National Weather Service (NWS), the Federal Aviation Administration (FAA), and the U.S. Air Force. The NEXRAD system detects precipitation and wind, and its data can be processed to map precipitation patterns and movement. NCEI provides access to archived NEXRAD Level-II data and Level-III products. [source](https://www.ncei.noaa.gov/products/radar/next-generation-weather-radar)

The NEXRAD data is freely available on the Amazon cloud at [https://registry.opendata.aws/noaa-nexrad/](https://registry.opendata.aws/noaa-nexrad/).

The data can be accessed using the Python ARM Radar Toolkit ([Py-ART](https://arm-doe.github.io/pyart/)), an open source library for working with weather radar data. Py-ART is partly supported by the U.S. Department of Energy as part of the Atmospheric Radiation Measurement (ARM) Climate Research Facility, an Office of Science user facility. Which was developed by JJ Helmus and SM Collis (JORS 2016, doi: 10.5334/jors.119)

[PyArt examples](https://projectpythia.org/radar-cookbook/notebooks/foundations/pyart-basics.html)

Here we use PyArt to find recent files from the Miami NEXRAD station and look at Miami to see if it is raining.

Just in case it is interesting, this will save 7200 rain data. As new rain comes in, old rain will fall off so it will remain a file with 7200 points. The last data point will be the most recent rain rate.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tempfile
import pickle
import pytz
from datetime import datetime
import pyart
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
import pandas as pd
import nexradaws

#import configuration location and filepath
from myconfig import *

output_path = output_path_rain_data
templocation = tempfile.mkdtemp()


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
conn = nexradaws.NexradAwsInterface()

In [3]:
from datetime import datetime, timedelta
# Get today's date
date_end = datetime.today()
date_start = date_end - timedelta(hours=24*7)
lyr_end,imon_end,idym_end,ihr_end = str(date_end.year), str(date_end.month), str(date_end.day),str(date_end.hour)

In [4]:
# get previous data
#fname = output_path + 'NEXRAD_scans.pkl'
#with open(fname, 'rb') as file:
#    old_scans = pickle.load(file)
fname = output_path + 'NEXRAD_rain.nc'
with xr.open_dataset(fname) as ds_all:
    print('opened nc and scan files')

opened nc and scan files


In [5]:
west_timezone = pytz.timezone('America/Los_Angeles')
start = west_timezone.localize(date_start) #datetime(2013,5,31,17,0))
end = west_timezone.localize (date_end) #datetime(2013,5,31,19,0))
scans = conn.get_avail_scans(2024,10,22, radar_id)
# Remove items containing '_MDM'
print(radar_id)
print(start,end)
#scans = conn.get_avail_scans_in_range(start, end, radar_id)
scans = [item for item in scans if '_MDM' not in str(item)]
print("There are {} scans available between {} and {}\n".format(len(scans), start, end))
ilen = len(scans)
print(radar_id,ilen)
if ilen>10:
    scans = scans[-10:-1]
#print(ilen)
#remove any repeat scans that were already downloaded
#scans = [item for item in scans if item.key not in [i.key for i in old_scans]]
print(len(scans))
#scans = scans[-10:-1]

KAMX
2024-10-19 00:00:57.867522-07:00 2024-10-26 00:00:57.867522-07:00
There are 336 scans available between 2024-10-19 00:00:57.867522-07:00 and 2024-10-26 00:00:57.867522-07:00

KAMX 336
9


# find rain
Where the clutter_filter_power_removed is >20 it is usually rain.

In [6]:
results = conn.download(scans, templocation)
for i,scan in enumerate(results.iter_success(),start=1):
    radar = scan.open_pyart()
    ds = pyart.util.columnsect.get_field_location(radar, site_lat, site_lon)
    # Apply the conditional operation on var2 based on var1
    ds = ds.drop_vars({'spectrum_width','differential_phase',
                       'differential_reflectivity','cross_correlation_ratio',
                       'time_offset','velocity','reflectivity'})
    ds = ds.rename({'base_time':'time'})
    ds = ds.set_coords('time')
    del ds['time'].attrs['units']
    if i==1:
        ds_all = ds.isel(height=0)
    else:
        ds_all = xr.concat([ds_all, ds.isel(height=0)],dim = 'time')

Downloaded KAMX20241022_233355_V06
Downloaded KAMX20241022_232936_V06
Downloaded KAMX20241022_233814_V06
Downloaded KAMX20241022_232058_V06
Downloaded KAMX20241022_232517_V06
Downloaded KAMX20241022_234233_V06


Downloaded KAMX20241022_234652_V06
Downloaded KAMX20241022_235530_V06


Downloaded KAMX20241022_235111_V06
9 out of 9 files downloaded...0 errors


In [7]:
#aws_combined = old_scans + scans
#ilen = len(scans)
# Remove the old items
#aws_files_filtered = aws_files[ilen:]
#ds_all = ds_all.isel(time=slice(ilen, None))
# output netcdf data and the list of files that have been downloaded and read
#fname = output_path + 'NEXRAD_rain.nc'
#ds_all.to_netcdf(fname)
# Save the list of objects to a file using pickle
#fname = output_path + 'NEXRAD_scans.pkl'
#with open(fname, 'wb') as file:
#    pickle.dump(aws_combined, file)


In [8]:
# New grid with 7200 points
start_time = pd.to_datetime(ds_all.time[0].data).strftime('%Y-%m-%dT%H:%M:%S')
end_time = pd.to_datetime(ds_all.time[-1].data).strftime('%Y-%m-%dT%H:%M:%S')
new_time = pd.date_range(start=start_time, end=end_time, periods=7200).to_numpy()
ds_all_interpolated = ds_all.interp(time=new_time)
ds = ds_all_interpolated
# create rain binary on/off from reflectivity
#ds['rain'] = xr.where(ds['clutter_filter_power_removed'] < 20, 0, xr.where(ds['clutter_filter_power_removed'] >= 20, 1, ds['reflectivity'))
ds['rain_flag'] =ds['clutter_filter_power_removed']*0
ds['rain_flag'] = xr.where(ds['clutter_filter_power_removed'] > 40, 1, 0)
#normalize reflectivity
ds['clutter_filter_power_removed'] = ds.clutter_filter_power_removed/ds.clutter_filter_power_removed.max()

# Instead of having the rain history through the piece, have a binary rain/no rain
# based on the last downloaded rain value, so if the latest value says is is raining set the 
# entire flag to rain or vice versa
if ds.rain_flag[-1]==0:
    ds.rain_flag==0
else:
    ds.rain_flag==1

#fix up interp
ds['clutter_filter_power_removed'][0]=ds['clutter_filter_power_removed'][2]
ds['clutter_filter_power_removed'][1]=ds['clutter_filter_power_removed'][2]
ds.height[0]=ds.height[2]
ds.height[1]=ds.height[2]

In [9]:
#create dataframe with time, normalized reflectivity, rain binary on off
is_rain_df = pd.DataFrame({'time':ds.time,'reflectivity_normalized':ds.clutter_filter_power_removed.data,'rain': ds.rain_flag.data})
fname = output_path_nrt + 'nrt_raining.csv'
is_rain_df.to_csv(fname,index=False)